In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
f = open('words_dictionary.json')
words= json.load(f)
possible_words=[word.lower() for word in list(words.keys()) if len(word)==5]
with open('War_and_Peace.txt', 'r') as f:
     book_txt = f.read().split(' ')
dictionary={}
for i,word in enumerate(book_txt):
    if word.lower() in possible_words:
        if word.lower() in list(dictionary.keys()):
            dictionary[word.lower()]+=1
        else:
            dictionary[word.lower()]=1 
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [2]:
better_words_lst=list(dictionary.keys())
guess_lst=possible_words
guess_df=[list(word) for word in guess_lst]
guess_df=pd.DataFrame(guess_df)
guess_df.columns=['0','1','2','3','4']
guess_df['word']= guess_lst
guess_df_2=guess_df.copy()
guess_df_2['01ind']= guess_df_2['0']==guess_df_2['1']
guess_df_2['02ind']= guess_df_2['0']==guess_df_2['2']
guess_df_2['03ind']= guess_df_2['0']==guess_df_2['3']
guess_df_2['04ind']= guess_df_2['0']==guess_df_2['4']
guess_df_2['12ind']= guess_df_2['1']==guess_df_2['2']
guess_df_2['13ind']= guess_df_2['1']==guess_df_2['3']
guess_df_2['14ind']= guess_df_2['1']==guess_df_2['4']
guess_df_2['23ind']= guess_df_2['2']==guess_df_2['3']
guess_df_2['24ind']= guess_df_2['2']==guess_df_2['4']
guess_df_2['34ind']= guess_df_2['3']==guess_df_2['4']
guess_df_2['match_all']= guess_df_2.iloc[:, -10:-1].sum(axis=1)
fst_snd_guess=guess_df_2.loc[guess_df_2.match_all==0]
fst_snd_guess=fst_snd_guess['word'].tolist()

In [19]:
import itertools
p=['B','Y','G']
all_possible_combination=[list(p) for p in itertools.product(p,repeat=5) ]

In [38]:
def probability_calculation(word, combination, word_list):
    guess_df=[list(words) for words in word_list]
    guess_df=pd.DataFrame(guess_df)
    guess_df.columns=['0','1','2','3','4']
    guess_df['word']= word_list
    p_l={
    'a': [-1],
    'b': [-1],
    'c': [-1],
    'd': [-1],
    'e': [-1],
    'f': [-1],
    'g': [-1],
    'h': [-1],
    'i': [-1],
    'j': [-1],
    'k': [-1],
    'l': [-1],
    'm': [-1],
    'n': [-1],
    'o': [-1],
    'p': [-1],
    'q': [-1],
    'r': [-1],
    's': [-1],
    't': [-1],
    'u': [-1],
    'v': [-1],
    'w': [-1],
    'x': [-1],
    'y': [-1],
    'z': [-1],
    }
    word=word.lower()
    guess=list(word)
    for i,col in enumerate(combination):
        if col=='G':
            if p_l[guess[i]][0] != 'present':
                p_l[guess[i]][0]='present'
            p_l[guess[i]].append(['G',i])
        elif col=='Y':
            if p_l[guess[i]][0] != 'present':
                p_l[guess[i]][0]='present'
            p_l[guess[i]].append(['Y',i])
        else:
            if p_l[guess[i]][0] == 'present':
                p_l[guess[i]].append(['Y',i])
            else:
                p_l[guess[i]][0]='absent'
    yes_sir_list=[(k,v[1:]) for (k,v) in p_l.items() if v[0]=='present']
    absent_list= [(k,v[0]) for (k,v) in p_l.items() if v[0]=='absent']
    position_list=[]
    present_list= []
    for (k,lis) in yes_sir_list:
        num=len(lis)
        for i in range(num):
            if lis[i][0]== 'Y':
                present_list.append((k,lis[i][1]))
            else:
                position_list.append((k,lis[i][1]))
    for l,p in position_list:
        guess_df= guess_df.loc[guess_df[str(p)]==l]
    for l,q in present_list:
        guess_df= guess_df.loc[~(guess_df[str(q)]==l)]
        guess_df= guess_df.loc[(guess_df.loc[:,:]==l).sum(axis=1).astype('bool')]
    for l,_ in absent_list:
        guess_df=guess_df.loc[~(guess_df.loc[:,:]==l).sum(axis=1).astype('bool')]
    probability=guess_df.shape[0]/len(word_list)  
    info= probability*np.log2(1/probability) if probability != 0 else 0
    return (info,guess_df)

In [55]:
info=0
for combination in all_possible_combination:
    info += probability_calculation('weary',combination,guess_lst)[0]
print(info)

4.886172298302406


In [ ]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
inputs = guess_lst
def info_calculation(word,all_possible_combination=all_possible_combination):
    info=0
    for combination in all_possible_combination:
        info += probability_calculation(word,combination,guess_lst)[0]
    return info
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(info_calculation)(i) for i in inputs)

In [53]:
num_cores

16